In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import xgboost
import lightgbm as lgb
import catboost
from sklearn.model_selection import KFold

In [2]:
from utility.data_cleaning import *
from utility.data_preprocessing import *
from utility.model_building import *
from config.configuration import *

In [3]:
from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [4]:
df=pd.read_excel(input_data_path)
df

,ClaimNumber,LossDate,PolicyNumber,PolicyEntryDate,PolicyExpirationDate,ClaimLossDesc,ClaimStatus,Examiner,ClaimLOB,InsuredName,...,Stories,Comments,ConstructionTypeDesc,FormattedAddress,ZipCode,LossAmount,ExternalKey,SquareFootage,BuildingLimit,E2Value
0,NaN,2019-05-27 00:00:00.000,7150080,2019-10-29 08:21:23.337,2020-05-15 00:00:00.000,Hail damage to the building,Closed,rgibson,BOP Property,GLEN KEGLEY,...,1,ClassCode:65141,Frame,"1165 DATE ST\n\rFAIRBORN, OH 45324",45324,525.60,b4D464A07531B48AD9584C868B0CB8862,100,144000.00,NaN
1,2006487.0,2020-07-10 00:00:00.000,7176669,2020-11-04 12:54:20.113,2020-12-28 00:00:00.000,Storm damage to the roof,Closed,ppavlik,BOP Property,"zIngenuity Management, LLC",...,1,ClassCode:65146,JoistedMasonry,"5420 Lonsdale North\n\rColumbus, OH 43232",43232,NaN,b3D02D4AABF094ED085FC37A057DB1C6B,250,23040.00,NaN
2,2006487.0,2020-07-10 00:00:00.000,7176669,2020-11-04 12:54:20.113,2020-12-28 00:00:00.000,Storm damage to the roof,Closed,ppavlik,BOP Property,"zIngenuity Management, LLC",...,1,ClassCode:65146,JoistedMasonry,"5420 Lonsdale North\n\rColumbus, OH 43232",43232,NaN,b3D02D4AABF094ED085FC37A057DB1C6B,250,23040.00,NaN
3,2007590.0,2021-07-10 00:00:00.000,5080437,2021-07-14 10:23:58.490,2021-10-26 00:00:00.000,A fire near the property caused damage to the ...,Closed,swright,BOP Property,IRIS COMMONS ASSOCIATES,...,2,ClassCode:65144,Frame,"618 CENTER STREET\n\rWILLIAMSPORT, PA 17701",17701,2265.00,b2415B7E5759242AEB5B62C3910BD65C1,300,60786.00,NaN
4,2005124.0,2020-01-31 00:00:00.000,643744,2020-02-03 09:58:15.567,2020-11-20 00:00:00.000,Fire,Closed,ahart,BOP Property,"Octavia Hill Association, Inc.",...,12,ClassCode:65144,JoistedMasonry,"1929 S. BEECHWOOD STREET\n\rPHILADELPHIA, PA 1...",19145,146441.35,b65CC72090C3640BABCC1040285C3DF32,380,41081.04,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,2010355.0,2022-01-15 00:00:00.000,7156220,2022-01-20 13:24:04.913,2023-01-01 00:00:00.000,Water intrusion from boiler which ran into ele...,Closed,bbrent,BOP Property,Amurcon Corp of Virginia,...,8,ClassCode:65132,MasonryNonCombustible,"1021 German School Road\n\rRichmond, VA 23225",23225,NaN,b42F07BB3D0EA472F8168C739BB51FD49,182744,26195067.00,NaN
6311,2008366.0,2022-01-15 00:00:00.000,7156220,2022-01-20 13:24:04.913,2023-01-01 00:00:00.000,Water pipe burst causing water damage.,Closed,ssexton,BOP Property,Amurcon Corp of Virginia,...,8,ClassCode:65132,MasonryNonCombustible,"1021 German School Road\n\rRichmond, VA 23225",23225,31154.38,b42F07BB3D0EA472F8168C739BB51FD49,182744,26195067.00,NaN
6312,2008366.0,2022-01-15 00:00:00.000,7156220,2022-01-20 13:24:04.913,2023-01-01 00:00:00.000,Water pipe burst causing water damage.,Closed,ssexton,BOP Property,Amurcon Corp of Virginia,...,8,ClassCode:65132,MasonryNonCombustible,"1021 German School Road\n\rRichmond, VA 23225",23225,31154.38,b42F07BB3D0EA472F8168C739BB51FD49,182744,26195067.00,NaN
6313,2010469.0,2023-02-20 00:00:00.000,7156220,2023-04-05 13:05:49.947,2024-01-01 00:00:00.000,Resident in unit #213 had liquid dripping in l...,Closed,jeshbach,BOP Property,Amurcon Corp of Virginia,...,8,ClassCode:65132,MasonryNonCombustible,"1021 German School Road\n\rRichmond, VA 23225",23225,NaN,b42F07BB3D0EA472F8168C739BB51FD49,182744,30124327.05,NaN


## for null E2Value fill data from predefined model

In [19]:
with open('./data/training/output/E2val_pred_lr_model_20231213.pkl', 'rb') as file:
    model=pickle.load(file)

In [5]:
# with open('./data/training/output/E2val_pred_xgr_model_20231214.pkl', 'rb') as file:
#     xgr=pickle.load(file)

In [7]:
df_new1 = df[df['E2Value'].isnull()]
df_new2 = df[df['E2Value'].notnull()]
from datetime import datetime
df_new1['YearGap'] = datetime.now().year - df_new1['YearBuilt']
predicted_e2value= model.predict(df_new1[['BuildingLimit', 'YearGap']])
#predicted_e2value= xgr.predict(df_new1[['BuildingLimit', 'YearGap']])
df_new1['new_e2val'] = predicted_e2value

C:\Users\SmrutisikhaJena\AppData\Local\Temp\ipykernel_17472\3466799862.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new1['YearGap'] = datetime.now().year - df_new1['YearBuilt']
C:\Users\SmrutisikhaJena\AppData\Local\Temp\ipykernel_17472\3466799862.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new1['new_e2val'] = predicted_e2value


In [8]:
df_new1 = df_new1.drop(columns=['E2Value'])
df_new1 = df_new1.rename(columns={'new_e2val': 'E2Value'})

In [9]:
columns=['ClaimNumber','PolicyNumber', 'Jurisdiction', 'CoverageType', 'Deductible', 'Limit', 'InjuryCause',
                  'YearBuilt', 'Stories', 'Comments', 'ConstructionTypeDesc', 'ZipCode', 'LossAmount','SquareFootage','BuildingLimit','E2Value']
df_new1 = df_new1[columns]
df_new2 = df_new1[columns]
df_final=pd.concat([df_new1, df_new2], ignore_index=True)
df_final = df_final.drop_duplicates()
df_final

,ClaimNumber,PolicyNumber,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,LossAmount,SquareFootage,BuildingLimit,E2Value
0,NaN,7150080,Ohio,BOP-Building,FixedDollar - 2500.00,Standard - 144000.00,HAIL,1957,1,ClassCode:65141,Frame,45324,525.60,100,144000.00,3.135326e+05
1,2006487.0,7176669,Ohio,BOP-Building,FixedDollar - 25000.00,Standard - 23040.00,WIND,1971,1,ClassCode:65146,JoistedMasonry,43232,NaN,250,23040.00,1.439233e+05
2,2006487.0,7176669,Ohio,BOP-Building,FixedDollar - 25000.00,Blanket Group 1 Blanket - 49858139.00,WIND,1971,1,ClassCode:65146,JoistedMasonry,43232,NaN,250,23040.00,1.439233e+05
3,2007590.0,5080437,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 60786.00,FIRE,2007,2,ClassCode:65144,Frame,17701,2265.00,300,60786.00,1.651150e+05
4,2005124.0,643744,Pennsylvania,BOP-Building,FixedDollar - 5000.00,Standard - 4108104.00,FIRE,1990,12,ClassCode:65144,JoistedMasonry,19145,146441.35,380,41081.04,1.398818e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084,2010355.0,7156220,Virginia,BOP-Building,FixedDollar - 10000.00,Blanket Group 1 Blanket - 379762361.00,WATER,1978,8,ClassCode:65132,MasonryNonCombustible,23225,NaN,182744,26195067.00,2.761131e+06
6085,2008366.0,7156220,Virginia,BOP-Building,FixedDollar - 10000.00,Standard - 26195066.00,WATER,1978,8,ClassCode:65132,MasonryNonCombustible,23225,31154.38,182744,26195067.00,2.761131e+06
6086,2008366.0,7156220,Virginia,BOP-Building,FixedDollar - 10000.00,Blanket Group 1 Blanket - 379762361.00,WATER,1978,8,ClassCode:65132,MasonryNonCombustible,23225,31154.38,182744,26195067.00,2.761131e+06
6087,2010469.0,7156220,Virginia,BOP-Building,FixedDollar - 10000.00,Standard - 30124327.00,OTHER PERILS,1978,8,ClassCode:65132,MasonryNonCombustible,23225,NaN,182744,30124327.05,2.761131e+06


In [10]:
df_final = df_final[selected_columns]
df_clean= clean_data(df_final)
df_clean


,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,LossAmount,E2Value,SquareFootage,BuildingLimit
0,ohio,bop-building,fixeddollar - 2500.00,standard - 144000.00,hail,1957,1,classcode:65141,frame,45324,525.60,3.135326e+05,100,144000.00
1,ohio,bop-building,fixeddollar - 25000.00,standard - 23040.00,wind,1971,1,classcode:65146,joistedmasonry,43232,NaN,1.439233e+05,250,23040.00
2,ohio,bop-building,fixeddollar - 25000.00,blanket group 1 blanket - 49858139.00,wind,1971,1,classcode:65146,joistedmasonry,43232,NaN,1.439233e+05,250,23040.00
3,pennsylvania,bop-building,fixeddollar - 1000.00,standard - 60786.00,fire,2007,2,classcode:65144,frame,17701,2265.00,1.651150e+05,300,60786.00
4,pennsylvania,bop-building,fixeddollar - 5000.00,standard - 4108104.00,fire,1990,12,classcode:65144,joistedmasonry,19145,146441.35,1.398818e+05,380,41081.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084,virginia,bop-building,fixeddollar - 10000.00,blanket group 1 blanket - 379762361.00,water,1978,8,classcode:65132,masonrynoncombustible,23225,NaN,2.761131e+06,182744,26195067.00
6085,virginia,bop-building,fixeddollar - 10000.00,standard - 26195066.00,water,1978,8,classcode:65132,masonrynoncombustible,23225,31154.38,2.761131e+06,182744,26195067.00
6086,virginia,bop-building,fixeddollar - 10000.00,blanket group 1 blanket - 379762361.00,water,1978,8,classcode:65132,masonrynoncombustible,23225,31154.38,2.761131e+06,182744,26195067.00
6087,virginia,bop-building,fixeddollar - 10000.00,standard - 30124327.00,other perils,1978,8,classcode:65132,masonrynoncombustible,23225,NaN,2.761131e+06,182744,30124327.05


      Jurisdiction  CoverageType   InjuryCause  Stories        ClassCode  \
0             ohio  bop-building          hail        1  classcode:65141   
1             ohio  bop-building          wind        1  classcode:65146   
2             ohio  bop-building          wind        1  classcode:65146   
3     pennsylvania  bop-building          fire        2  classcode:65144   
4     pennsylvania  bop-building          fire       12  classcode:65144   
...            ...           ...           ...      ...              ...   
6084      virginia  bop-building         water        8  classcode:65132   
6085      virginia  bop-building         water        8  classcode:65132   
6086      virginia  bop-building         water        8  classcode:65132   
6087      virginia  bop-building  other perils        8  classcode:65132   
6088      virginia  bop-building  other perils        8  classcode:65132   

       ConstructionTypeDesc ZipCode  LossAmount       E2Value  SquareFootage  \
0      

,Jurisdiction,CoverageType,InjuryCause,Stories,ClassCode,ConstructionTypeDesc,ZipCode,LossAmount,E2Value,SquareFootage,BuildingLimit,limit_type,Limit_val,Deductible_type,Deductible_val,YearGap
0,ohio,bop-building,hail,1,classcode:65141,frame,45324,525.60,3.135326e+05,100,144000,standard,144000.0,fixeddollar,2500,66
1,ohio,bop-building,wind,1,classcode:65146,joistedmasonry,43232,NaN,1.439233e+05,250,23040,standard,23040.0,fixeddollar,25000,52
2,ohio,bop-building,wind,1,classcode:65146,joistedmasonry,43232,NaN,1.439233e+05,250,23040,blanket,1.0,fixeddollar,25000,52
3,pennsylvania,bop-building,fire,2,classcode:65144,frame,17701,2265.00,1.651150e+05,300,60786,standard,60786.0,fixeddollar,1000,16
4,pennsylvania,bop-building,fire,12,classcode:65144,joistedmasonry,19145,146441.35,1.398818e+05,380,41081,standard,4108104.0,fixeddollar,5000,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6084,virginia,bop-building,water,8,classcode:65132,masonrynoncombustible,23225,NaN,2.761131e+06,182744,26195067,blanket,1.0,fixeddollar,10000,45
6085,virginia,bop-building,water,8,classcode:65132,masonrynoncombustible,23225,31154.38,2.761131e+06,182744,26195067,standard,26195066.0,fixeddollar,10000,45
6086,virginia,bop-building,water,8,classcode:65132,masonrynoncombustible,23225,31154.38,2.761131e+06,182744,26195067,blanket,1.0,fixeddollar,10000,45
6087,virginia,bop-building,other perils,8,classcode:65132,masonrynoncombustible,23225,NaN,2.761131e+06,182744,30124327,standard,30124327.0,fixeddollar,10000,45


In [10]:
df_clean['InjuryCause'].value_counts(dropna=False)

InjuryCause
water                             2473
wind                              1091
fire                               669
other perils                       574
hail                               304
vehicles                           279
falling objects                    117
vandalism & malicious mischief     104
freezing                            99
lightning                           57
theft                               46
collapse                            41
mold                                40
glass                               34
weight ice/snow/sleet               29
collision                           26
electric current                    26
smoke                               18
<NA>                                12
burglary                            11
cracking/rupture                    10
sinkhole                             9
bodily injury                        5
contamination/ pollution             5
discrimination                       5
flood        

In [11]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6089 entries, 0 to 6088
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Jurisdiction          6089 non-null   string 
 1   CoverageType          6089 non-null   string 
 2   InjuryCause           6077 non-null   string 
 3   Stories               6089 non-null   int32  
 4   ClassCode             6082 non-null   string 
 5   ConstructionTypeDesc  6089 non-null   string 
 6   ZipCode               6089 non-null   string 
 7   LossAmount            4517 non-null   float64
 8   E2Value               6089 non-null   float64
 9   SquareFootage         6089 non-null   int32  
 10  BuildingLimit         6089 non-null   int32  
 11  limit_type            6089 non-null   string 
 12  Limit_val             6089 non-null   float64
 13  Deductible_type       6089 non-null   string 
 14  Deductible_val        6089 non-null   int32  
 15  YearGap               6089

In [12]:
print(df_clean.isnull().sum())

Jurisdiction               0
CoverageType               0
InjuryCause               12
Stories                    0
ClassCode                  7
ConstructionTypeDesc       0
ZipCode                    0
LossAmount              1572
E2Value                    0
SquareFootage              0
BuildingLimit              0
limit_type                 0
Limit_val                  0
Deductible_type            0
Deductible_val             0
YearGap                    0
dtype: int64


In [11]:
X_train, y_train, X_test, y_test= preprocess_and_save(df_clean)

C:\Users\SmrutisikhaJena\Documents\GitHub\Risk_Predictive_Model\utility\data_preprocessing.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LossAmount'] = df['LossAmount'].apply(extract_amount)


In [12]:

train_and_evaluate_model(X_train, y_train, X_test, y_test)

Linear Regression
220983.79747090687
0.032622939192129885
XGBoost
150091.72772163668
0.5537395143204744
RF
155073.56401383106
0.5236233603115936
CATBoost
215870.21786546506
0.0768752825182294
cbr_Kfold
80138.84353225742
0.10297128913212739
xgb_Kfold
101530.99826173097
-0.43985087341829443
 rf_model
109070.82009749883
-0.6616416093905584
ensemble
83285.26040348431
0.03115010251003425


(155073.56401383106, 0.5236233603115936)